# [0.1] - Ray Tracing

Colab: [exercises](https://colab.research.google.com/drive/1L-N_B9SyE8v2jiAsLjOMyjfogyvTtXaU) | [solutions](https://colab.research.google.com/drive/1vBjSYNahPqzGfxEaKx4ZEEGKswRRcUvo)

ARENA 3.0 [Streamlit page](https://arena3-chapter0-fundamentals.streamlit.app/[0.1]_Ray_Tracing)

Please send any problems / bugs on the `#errata` channel in the [Slack group](https://join.slack.com/t/arena-la82367/shared_invite/zt-1uvoagohe-JUv9xB7Vr143pdx1UBPrzQ), and ask any questions on the dedicated channels for this chapter of material.



<img src="https://raw.githubusercontent.com/callummcdougall/Fundamentals/main/images/raytracing.png" width="350">


# Introduction & Setup


Today we'll be practicing batched matrix operations in PyTorch by writing a basic graphics renderer. We'll start with an extremely simplified case and work up to rendering your very own 3D Pikachu!

We'll also be touching on some general topics which will be important going forwards in this course, such as:

* Using GPT systems to assist your learning and coding
* Typechecking, and good coding practices
* Debugging, with VSCode's built-in run & debug features


## Getting started

As a reminder, the [GitHub repo](https://github.com/callummcdougall/ARENA_3.0) which hosts the exercises pages (and the exercises) looks something like this:

```
.
├── chapter0_fundamentals
│   ├── exercises
│   │   ├── part0_prereqs
│   │   ├── part1_ray_tracing
│   │   │   ├── solutions.py
│   │   │   ├── tests.py
│   │   ├── part2_cnns
│   │   ⋮    ⋮
│   └── instructions
│       └── Home.py
├── chapter1_transformers
├── chapter2_rl
└── requirements.txt
```

When you run the setup code below, you'll clone the repository into your Colab's local storage, which is how the imports will work. However, you should be working only from this Colab notebook - filling in functions where you are asked to. Each exercise is denoted with a little box that rates difficulty & importance of the exercise. Most exercises will also have test functions which can be run, to check your solution works (often these test functions will compare your function's output to the output of the function in `solutions.py`).

Lastly, each exercise will have solutions below it in a dropdown, in case you get stuck.

## Setup

Run these cells below (don't worry about reading through them).


In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Install packages
    %pip install einops
    %pip install jaxtyping
    %pip install typeguard

    # Code to make sure output widgets display
    from google.colab import output
    output.enable_custom_widget_manager()

    # Code to download the necessary files (e.g. solutions, test funcs)
    import os, sys
    if not os.path.exists("chapter0_fundamentals"):
        !wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
        !unzip /content/main.zip 'ARENA_3.0-main/chapter0_fundamentals/exercises/*'
        os.remove("/content/main.zip")
        os.rename("ARENA_3.0-main/chapter0_fundamentals", "chapter0_fundamentals")
        os.rmdir("ARENA_3.0-main")
        sys.path.insert(0, "chapter0_fundamentals/exercises")

    # Clear output
    from IPython.display import clear_output
    clear_output()
    print("Imports & installations complete!")

else:
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

In [3]:
import os, sys
from pathlib import Path

current_dir = Path(os.getcwd())
plotly_utils_dir = current_dir.parent
if str(plotly_utils_dir) not in sys.path:
    sys.path.append(str(plotly_utils_dir))

In [47]:
import os
import sys
import torch as t
from torch import Tensor
import einops
from ipywidgets import interact
import plotly.express as px
from ipywidgets import interact
from pathlib import Path
from IPython.display import display
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
import typeguard

# Get file paths to this set of exercises
exercises_dir = Path("chapter0_fundamentals/exercises")
section_dir = exercises_dir / "part1_ray_tracing"

from plotly_utils import imshow
from part1_ray_tracing.utils import render_lines_with_plotly, setup_widget_fig_ray, setup_widget_fig_triangle
import part1_ray_tracing.tests as tests

MAIN = __name__ == "__main__"

<details>
<summary>Help - I get a NumPy-related error</summary>

This is an annoying colab-related issue which I haven't been able to find a satisfying fix for. If you restart runtime (but don't delete runtime), and run just the imports cell above again (but not the `%pip install` cell), the problem should go away.
</details>


<details>
<summary>Note on <code>pathlib</code></summary>

We'll be using the `pathlib` library to define file paths. This is a more modern way of working with file paths than the `os` library, and is more cross-platform. You can read more about it [here](https://realpython.com/python-pathlib/).

A major advantage of using `pathlib` rather than just relative path names is that it is more robust to which file / directory you happen to be running your code in. There's nothing more frustrating than failing to import or load a file, even though you can see it right there in your directory! Most of our code to load files will look something like this:

```python
with open(section_dir / "pikachu.pt", "rb") as f:
    triangles = t.load(f)
```

since `section_dir` is the name of the `part1_ray_tracing` directory, and forward slashes are used to define files or directories within that directory.
</details>


In [5]:
def first_func(num_pixels, y_limit) -> t.Tensor:
    rays = t.zeros((num_pixels, 2, 3), dtype=t.float32)
    t.linspace(-y_limit, y_limit, num_pixels, out=rays[:, 1, 1])
    rays[:, 1, 0] = 1
    return rays

segments = t.tensor([
    [[1.0, -12.0, 0.0], [1, -6.0, 0.0]],
    [[0.5, 0.1, 0.0], [0.5, 1.15, 0.0]],
    [[2, 12.0, 0.0], [2, 21.0, 0.0]]
])

rays1d = first_func(9, 10.0)

# Exercises

## 1D Image Rendering

In our initial setup, the **camera** will be a single point at the origin, and the **screen** will be the plane at x=1.

**Objects** in the world consist of triangles, where triangles are represented as 3 points in 3D space (so 9 floating point values per triangle). You can build any shape out of sufficiently many triangles and your Pikachu will be made from 412 triangles.

The camera will emit one or more **rays**, where a ray is represented by an **origin** point and a **direction** point. Conceptually, the ray is emitted from the origin and continues in the given direction until it intersects an object.

We have no concept of lighting or color yet, so for now we'll say that a pixel on our screen should show a bright color if a ray from the origin through it intersects an object, otherwise our screen should be dark.


<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/ray_tracing.png" width="400">


To start, we'll let the z dimension in our `(x, y, z)` space be zero and work in the remaining two dimensions.


#### Exercise - implement `make_rays_1d`

```yaml
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```


Implement the following `make_rays_1d` function so it generates some rays coming out of the origin, which we'll take to be `(0, 0, 0)`.

Calling `render_lines_with_plotly` on your rays will display them in a 3D plot.


In [101]:
def make_rays_1d(num_pixels: int, y_limit: float) -> t.Tensor:
    '''
    num_pixels: The number of pixels in the y dimension. Since there is one ray per pixel, this is also the number of rays.
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both endpoints.

    Returns: shape (num_pixels, num_points=2, num_dim=3) where the num_points dimension contains (origin, direction) and the num_dim dimension contains xyz.

    Example of make_rays_1d(9, 1.0): [
        [[0, 0, 0], [1, -1.0, 0]],
        [[0, 0, 0], [1, -0.75, 0]],
        [[0, 0, 0], [1, -0.5, 0]],
        ...
        [[0, 0, 0], [1, 0.75, 0]],
        [[0, 0, 0], [1, 1, 0]],
    ]
    '''
    tensor = t.zeros((num_pixels, 2, 3))
    tensor[:,1,0] = 1
    step_size = (2 * y_limit) / (num_pixels - 1)
    pixels = t.arange(-y_limit, y_limit + step_size, step_size)
    tensor[:,1,1] = pixels
    return tensor

# num pixels by 2 by 3
# 0001_0 

rays1d = make_rays_1d(9, 10.0)
fig = render_lines_with_plotly(rays1d)

### Tip - the `out` keyword argument

Many PyTorch functions take an optional keyword argument `out`. If provided, instead of allocating a new tensor and returning that, the output is written directly to the `out` tensor.

If you used `torch.arange` or `torch.linspace` above, try using the `out` argument. Note that a basic indexing expression like `rays[:, 1, 1]` returns a view that shares storage with `rays`, so writing to the view will modify `rays`. You'll learn more about views later today.

## Ray-Object Intersection

Suppose we have a line segment defined by points $L_1$ and $L_2$. Then for a given ray, we can test if the ray intersects the line segment like so:

- Supposing both the ray and line segment were infinitely long, solve for their intersection point.
- If the point exists, check whether that point is inside the line segment and the ray.

Our camera ray is defined by the origin $O$ and direction $D$ and our object line is defined by points $L_1$ and $L_2$.

We can write the equations for all points on the camera ray as $R(u)=O +u D$ for $u \in [0, \infty)$ and on the object line as $O(v)=L_1+v(L_2 - L_1)$ for $v \in [0, 1]$.

The following interactive widget lets you play with this parameterization of the problem. Run the cells one after another:


In [14]:
fig = setup_widget_fig_ray()
display(fig)

@interact
def response(seed=(0, 10, 1), v=(-2.0, 2.0, 0.01)):
    t.manual_seed(seed)
    L_1, L_2 = t.rand(2, 2)
    P = lambda v: L_1 + v * (L_2 - L_1)
    x, y = zip(P(-2), P(2))
    with fig.batch_update():
        fig.data[0].update({"x": x, "y": y})
        fig.data[1].update({"x": [L_1[0], L_2[0]], "y": [L_1[1], L_2[1]]})
        fig.data[2].update({"x": [P(v)[0]], "y": [P(v)[1]]})

FigureWidget({
    'data': [{'type': 'scatter', 'uid': 'f3d6382f-e0e4-4656-a53d-4d842393759b', 'x': [], 'y': []},
             {'marker': {'size': 12},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'e49ca2b7-b264-460d-b407-5d7cc545ec96',
              'x': [],
              'y': []},
             {'marker': {'size': 12, 'symbol': 'x'},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'cc1d5f36-ec98-4c36-9252-783f32ce1ca5',
              'x': [],
              'y': []}],
    'layout': {'height': 500,
               'showlegend': False,
               'template': '...',
               'width': 600,
               'xaxis': {'range': [-1.5, 2.5]},
               'yaxis': {'range': [-1.5, 2.5]}}
})

interactive(children=(IntSlider(value=5, description='seed', max=10), FloatSlider(value=0.0, description='v', …

Setting the line equations from above equal gives the solution:

$$
\begin{aligned}O + u D &= L_1 + v(L_2 - L_1) \\ u D - v(L_2 - L_1) &= L_1 - O  \\ \begin{pmatrix} D_x & (L_1 - L_2)_x \\ D_y & (L_1 - L_2)_y \\ \end{pmatrix} \begin{pmatrix} u \\ v \\ \end{pmatrix} &=  \begin{pmatrix} (L_1 - O)_x \\ (L_1 - O)_y \\ \end{pmatrix} \end{aligned}
$$

Once we've found values of $u$ and $v$ which satisfy this equation, if any (the lines could be parallel) we just need to check that $u \geq 0$ and $v \in [0, 1]$.


### Exercise - which segments intersect with the rays?

```yaml
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵⚪⚪⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

For each of the following segments, which camera rays from earlier intersect? You can do this by inspection or using `render_lines_with_plotly`.


In [15]:
segments = t.tensor([
    [[1.0, -12.0, 0.0], [1, -6.0, 0.0]],
    [[0.5, 0.1, 0.0], [0.5, 1.15, 0.0]],
    [[2, 12.0, 0.0], [2, 21.0, 0.0]]
])

### Exercise - implement `intersect_ray_1d`

```yaml
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 20-25 minutes on this exercise.

It involves some of today's core concepts: tensor manipulation, linear operations, etc.
```

Using [`torch.lingalg.solve`](https://pytorch.org/docs/stable/generated/torch.linalg.solve.html) and [`torch.stack`](https://pytorch.org/docs/stable/generated/torch.stack.html), implement the `intersect_ray_1d` function to solve the above matrix equation.

<details>
<summary>Aside - difference between stack and concatenate</summary>

`torch.stack` will combine tensors along a new dimension.

```python
>>> t.stack([t.ones(2, 2), t.zeros(2, 2)], dim=0)
tensor([[[1., 1.],
         [1., 1.]],

        [[0., 0.],
         [0., 0.]]])
```

`torch.concat` (alias `torch.cat`) will combine tensors along an existing dimension.

```python
>>> t.cat([t.ones(2, 2), t.zeros(2, 2)], dim=0)
tensor([[1., 1.],
        [1., 1.],
        [0., 0.],
        [0., 0.]])
```

Here, you should use `torch.stack` to construct e.g. the matrix on the left hand side, because you want to combine the vectors $D$ and $L_1 - L_2$ to make a matrix.
</details>

Is it possible for the solve method to fail? Give a sample input where this would happen.

<details>
<summary>Answer - Failing Solve</summary>

If the ray and segment are exactly parallel, then the solve will fail because there is no solution to the system of equations. For this function, handle this by catching the exception and returning False.
</details>


In [59]:
# def intersect_ray_1d(ray: t.Tensor, segment: t.Tensor) -> bool:
#     '''
#     ray: shape (n_points=2, n_dim=3)  # O, D points
#     segment: shape (n_points=2, n_dim=3)  # L_1, L_2 points

#     Return True if the ray intersects the segment.
#     '''
#     lhs = t.stack([ray[1][:2], segment[0][:2] - segment[1][:2]], dim=1)
#     rhs = segment[0][:2] - ray[0][:2]
#     rhs = einops.rearrange(rhs, "h -> h 1")
#     try:
#         x = t.linalg.solve(lhs, rhs)
#     except Exception as e:
#         return False
#     return x[0] >= 0 and 0 <= x[1] <= 1

@jaxtyped
@typeguard.typechecked
def intersect_ray_1d(ray: Float[Tensor, "2 3"], segment: Float[Tensor, "2 3"]) -> bool:
    '''
    ray: shape (n_points=2, n_dim=3)  # O, D points
    segment: shape (n_points=2, n_dim=3)  # L_1, L_2 points

    Return True if the ray intersects the segment.
    '''
    lhs = t.stack([ray[1][:2], segment[0][:2] - segment[1][:2]], dim=1)
    rhs = segment[0][:2] - ray[0][:2]
    rhs = einops.rearrange(rhs, "h -> h 1")
    try:
        x = t.linalg.solve(lhs, rhs)
    except Exception as e:
        return False
    return bool((x[0] >= 0 and 0 <= x[1] <= 1)[0])

tests.test_intersect_ray_1d(intersect_ray_1d)
tests.test_intersect_ray_1d_special_case(intersect_ray_1d)

All tests in `test_intersect_ray_1d` passed!
All tests in `test_intersect_ray_1d_special_case` passed!


<details>
<summary>Help! My code is failing with a 'must be batches of square matrices' exception.</summary>

Our formula only uses the x and y coordinates - remember to discard the z coordinate for now.

It's good practice to write asserts on the shape of things so that your asserts will fail with a helpful error message. In this case, you could assert that the `mat` argument is of shape (2, 2) and the `vec` argument is of shape (2,). Also, see the aside below on typechecking.
</details>


### Aside - typechecking




> *Note - this section was written with VSCode in mind. The formatting for this kind of typechecking doesn't seem to work very well in Colab, and strict typechecking of the kind described below also won't work (which is why it's been removed). If you're interested in using these features, we'd recommend trying to set yourself up on VSCode.*


Typechecking is a useful habit to get into. It's not strictly necessary, but it can be a great help when you're debugging.

One good way to typecheck in PyTorch is with the `jaxtyping` library. In this library, we can use objects like `Float`, `Int`, `Bool`, etc object to specify the shape and data type of a tensor (or `Shaped` if we don't care about the data type).

In its simplest form, this just behaves like a fancier version of a docstring or comment (signalling to you, as well as any readers, what the size of objects should be). But you can also use the `typeguard` library to strictly enforce the type signatures of your inputs and outputs. For instance, consider the following typechecked function:


```python
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
import typeguard
from torch import Tensor

@jaxtyped
@typeguard.typechecked
def my_concat(x: Float[Tensor, "a1 b"], y: Float[Tensor, "a2 b"]) -> Float[Tensor, "a1+a2 b"]:
    return t.concat([x, y], dim=0)

x = t.ones(3, 2)
y = t.randn(4, 2)
z = my_concat(x, y)
```


This cell will run without error, because the tensor `t.concat([x, y], dim=0)` has shape `(3+4, 2) = (7, 2)`, which agrees with the symbolic representation of `(a1 b), (a2 b) -> (a1+a2, b)` in the type signature. But this code will fail if the type signatures are violated in any way, for instance:

* `x` or `y` are not 2D tensors
* The last dimension of `x` and `y` doesn't match
* The output doesn't have shape `(x.shape[0] + y.shape[0], x.shape[1])`
* `x`, `y` or the output are of type int rather than type float

You can test these out for yourself, by changing the cell above and then re-running it.

Jaxtyping has many other useful features, for example:

* Generic tensors can be represented with `Float[Tensor, "..."]`
* Tensors with a single scalar value can be represented with `Float[Tensor, ""]`
* Fixed dimensions can be represented with numbers, e.g. `Float[Tensor, "a b 4"]`
* Dimensions can be named *and* fixed, e.g. `x: Float[Tensor, "b=3"], y: Float[Tensor, "b"]` will raise an error if `x` and `y` don't *both* have shape `(3,)`
* You can even use these objects for inline assert statements, e.g. `assert isinstance(x, Float[Tensor, "3 b"])` asserts that `x` is a 2D tensor of type float, with first dimension `3`.

You can find more features of jaxtyping [here](https://docs.kidger.site/jaxtyping/).

Overall, type-checking is a really useful tool to have at your disposal, because it can help you quickly catch bugs in your code (and it helps your code be explicit and readable, both to possible collaborators / pair programming partners, and to your future self!).

We won't enforce strict typechecking in general, but you should feel free to use it when you're writing or using functions. (It's usually more useful when you're in the development stage, rather than once you're already confident that a function works correctly and will always receive the correct inputs.)


Exercise - can you rewrite the function `intersect_ray_1d` to use typechecking?

<details>
<summary>Answer</summary>

Your typechecked function might look like:

```python
@jaxtyped
@typeguard.typechecked
def intersect_ray_1d(ray: Float[Tensor, "points=2 dim=3"], segment: Float[Tensor, "points=2 dim=3"]) -> bool:
    '''
    ray: O, D points
    segment: L_1, L_2 points

    Return True if the ray intersects the segment.
    '''
```

This is if your solution just returned a boolean. If you left your boolean as a single-element PyTorch tensor, then a return type of `Bool[Tensor, ""]` would be appropriate.
</details>


## Batched Ray-Segment Intersection

Next, implement a batched version that takes multiple rays, multiple line segments, and returns a boolean for each ray indicating whether **any** segment intersects with that ray.

Note - in the batched version, we don't want the solver to throw an exception just because some of the equations don't have a solution - these should just return False.


### Tip - Ellipsis

You can use an ellipsis `...` in an indexing expression to avoid repeated `:` and to write indexing expressions that work on varying numbers of input dimensions.

For example, `x[..., 0]` is equivalent to `x[:, :, 0]` if `x` is 3D, and equivalent to `x[:, :, :, 0]` if `x` is 4D.

### Tip - Elementwise Logical Operations on Tensors

For regular booleans, the keywords `and`, `or`, and `not` are used to do logical operations and the operators `&`, `|`, and `~` do and, or and not on each bit of the input numbers. For example `0b10001 | 0b11000` is `0b11001` or 25 in base 10.

Tragically, Python doesn't allow classes to overload keywords, so if `x` and `y` are of type `torch.Tensor`, then `x and y` does **not** do the natural thing that you probably expect, which is compute `x[i] and y[i]` elementwise. It actually tries to coerce `x` to a regular boolean, which throws an exception.

As a workaround, PyTorch (and NumPy) have chosen to overload the bitwise operators but have them actually mean logical operations, since you usually don't care to do bitwise operations on tensors. So the correct expression would be `x & y` to compute `x[i] and y[i]` elementwise.

Another gotcha when it comes to logical operations - **operator precedence**. For instance, `v >= 0 & v <= 1` is actually evaluated as `(v >= (0 & v)) <= 1` (because `&` has high precedence). When in doubt, use parentheses to force the correct parsing: `(v >= 0) & (v <= 1)`.

### Tip - `einops`

Einops is a useful library which we'll dive deeper with tomorrow. For now, the only important function you'll need to know is `einops.repeat`. This takes as arguments a tensor and a string, and returns a new tensor which has been repeated along the specified dimensions. For example, the following code shows how we can repeat a 2D tensor along the last dimension:

```python
x = t.randn(2, 3)
x_repeated = einops.repeat(x, 'a b -> a b c', c=4)

assert x_repeated.shape == (2, 3, 4)
for c in range(4):
    t.testing.assert_close(x, x_repeated[:, :, c])
```

*(The function `t.testing.assert_close` checks that two tensors are the same shape, and all elements are the same value, within some very small numerical error.)*

### Tip - Logical Reductions

In plain Python, if you have a list of lists and want to know if any element in a row is `True`, you could use a list comprehension like `[any(row) for row in rows]`. The efficient way to do this in PyTorch is with `torch.any()` or equivalently the `.any()` method of a tensor, which accept the dimension to reduce over. Similarly, `torch.all()` or `.all()` method. Both of these methods accept a `dim` argument, which is the dimension to reduce over.

<details>
<summary>Aside - tensor methods</summary>

Most functions like `torch.any(tensor, ...)` (which take a tensor as first argument) have an equivalent tensor method `tensor.any(...)`. We'll see many more examples of functions like this as we go through the course.
</details>

### Tip - Broadcasting

Broadcasting is what happens when you perform an operation on two tensors, and one is a smaller size, but is copied along the dimensions of the larger one in order to apply to it. Below is an example (where `B` is copied along the first dimension of `A`):

```python
A = t.randn(2, 3)
B = t.randn(3)
AplusB = A + B

assert AplusB.shape == (2, 3)
for i in range(2):
    t.testing.assert_close(AplusB[i], A[i] + B)
```

Broadcasting sematics are a bit messy, and we'll go into it in more detail later in the course. If you want to get a full picture of it then click on the dropdown below, but for now here's the important thing to know - *dimensions get appended to the **start** of the smaller tensor `B`, and it is copied along those dimensions until its shape matches the larger tensor `A`*.

<details>
<summary>Aside - details of broadcasting</summary>

If you try to broadcast tensors `A` and `B`, then the following happens:

* The tensor with fewer dimensions is padded with dimensions of size one on the left.
* Once the tensors have the same number of dimensions, they are checked for compatibility.
    * Two dimensions are compatible if they are equal, or if one of them is one (in the latter case, we repeat the size-1 tensor along that dimension until it's the same size as the larger one).
    * If they are not compatible, then broadcasting is not allowed.

For instance, in the example above, first `B` is left-padded to shape `(1, 3)`, then it is copied along the first dimension to shape `(2, 3)`, when it can be added to `A`.

On the other hand, if `B` had shape `(2,)` then broadcasting would fail, because dimensions can't be added to the right of a tensor.
</details>


### Summary of all these tips

> * Use `...` to avoid repeated `:` in indexing expressions.
> * Use `&`, `|`, and `~` for elementwise logical operations on tensors.
> * Use parentheses to force the correct operator precedence.
> * Use `torch.any()` or `.any()` to do logical reductions (you can do this over a single dimension, with the `dim` argument).
> * If you're trying to broadcast tensors `A` and `B` (where `B` has fewer dimensions), this will work if the shape of `B` matches the ***last*** dimensions of `A` (in this case, `B` will get copied along the earlier dimensions of `A`).


### Exercise - implement `intersect_rays_1d`

```yaml
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 25-30 minutes on this exercise.

It will be one of the most difficult today; certainly it has the biggest step in difficulty.
```

With all these tips in mind, now you should implement `intersect_rays_1d`, the batched version of `intersect_ray_1d` (which returns True for each ray, if it intersects *any* segment).


In [99]:
@jaxtyped
@typeguard.typechecked
def intersect_rays_1d(rays: Float[Tensor, "nrays 2 3"], segments: Float[Tensor, "nsegments 2 3"]) -> Bool[Tensor, "nrays"]:
    '''
    For each ray, return True if it intersects any segment.
    '''
    rays_xy = rays[..., :2]
    segments_xy = segments[..., :2]
    col1 = rays_xy[:, 1, :]
    col2 = segments_xy[:, 0, :] - segments_xy[:, 1, :]
    col1 = einops.repeat(col1, 'nrays xy -> nrays nsegments xy', nsegments = segments.shape[0])
    col2 = einops.repeat(col2, 'nsegments xy -> nrays nsegments xy', nrays = rays.shape[0])
    lhs = t.stack([col1, col2], dim=3)
    col3 = segments_xy[:, 0, :]
    col4 = rays_xy[:, 0, :]
    col3 = einops.repeat(col3, 'nsegments xy -> nrays nsegments xy', nrays = rays.shape[0])
    col4 = einops.repeat(col4, 'nrays xy -> nrays nsegments xy', nsegments = segments.shape[0])
    rhs = col3 - col4
    dets = t.linalg.det(lhs)
    mask = dets.abs() <= 1e-10
    lhs[mask] = t.eye(2)
    xs = t.linalg.solve(lhs, rhs)
    output = (xs[...,0] >= 0) & (xs[...,1] >= 0) & (xs[...,1] <= 1)
    output[mask] = False
    return t.any(output, dim=1)

tests.test_intersect_rays_1d(intersect_rays_1d)
tests.test_intersect_rays_1d_special_case(intersect_rays_1d)

All tests in `test_intersect_rays_1d` passed!
All tests in `test_intersect_rays_1d_special_case` passed!


<details>
<summary>Help - I'm not sure how to implment this function without a for loop.</summary>

Initially, `rays.shape == (NR, 2, 3)` and `segments.shape == (NS, 2, 3)`. Try performing `einops.repeat` on them, so both their shapes are `(NR, NS, 2, 3)`. Then you can formulate and solve the batched system of matrix equations.
</details>

<details>
<summary>Help - I'm not sure how to deal with the cases of zero determinant.</summary>

You can use `t.linalg.det` to compute the determinant of a matrix, or batch of matrices *(gotcha: the determinant won't be exactly zero, but you can check that it's very close to zero, e.g. `det.abs() < 1e-6`)*. This will give you a boolean mask for which matrices are singular.

You can set all singular matrices to the identity (this avoids errors), and then at the very end you can use your boolean mask again to set the intersection to `False` for the singular matrices.
</details>

<details>
<summary>Help - I'm still stuck on the zero determinant cases.</summary>

After formulating the matrix equation, you should have a batch of matrices of shape `(NR, NS, 2, 2)`, i.e. `mat[i, j, :, :]` is a matrix which looks like:

$$
\begin{pmatrix} D_x & (L_1 - L_2)_x \\ D_y & (L_1 - L_2)_y \\ \end{pmatrix}
$$

Calling `t.linalg.det(mat)` will return an array of shape `(NR, NS)` containing the determinants of each matrix. You can use this to construct a mask for the singular matrices *(gotcha: the determinant won't be exactly zero, but you can check that it's very close to zero, e.g. `det.abs() < 1e-6`)*.

Indexing `mat` by this mask will return an array of shape `(x, 2, 2)`, where the zeroth axis indexes the singular matrices. As we discussed in the broadcasting section earlier, this means we can use broadcasting to set all these singular matrices to the identity:

```python
mat[is_singular] = t.eye(2)
```
</details>

## Using GPT to understand code


Next week we'll start learning about transformers and how to build them, but it's not too early to start using them to accelerate your own learning!

We'll be discussing more advanced ways to use GPT 3 and 4 as coding partners / research assistants in the coming weeks, but for now we'll look at a simple example: **using GPT to understand code**. You're recommended to read the recent [LessWrong post](https://www.lesswrong.com/posts/ptY6X3BdW4kgqpZFo/using-gpt-4-to-understand-code) by Siddharth Hiregowdara in which he explains his process. This works best on GPT-4, but I've found GPT-3.5 works equally well for reasonably straightforward problems (see the section below).

Firstly, you should get an account to use GPT with if you haven't already. Next, try asking GPT-3.5 / 4 for an explanation of the function above. You can do this e.g. via the following prompt:

```
Explain this Python function, line by line. You should break up your explanation by inserting sections of the code.

def intersect_rays_1d(rays: Float[Tensor, "nrays 2 3"], segments: Float[Tensor, "nsegments 2 3"]) -> Bool[Tensor, "nrays"]:
    NR = rays.size(0)
    NS = segments.size(0)
    rays = rays[..., :2]
    ...
```

I've found removing comments is often more helpful, because then GPT will answer in its own words rather than just repeating the comments (and the comments can sometimes confuse it).

Once you've got a response, here are a few more things you might want to consider asking:

* Can you suggest ways to improve the code?
    * GPT-4 recommended using a longer docstring and more descriptive variable names, among other things.
* Can you explain why the line `mat[is_singular] = t.eye(2)` works?
    * GPT-4 gave me a correct and very detailed explanation involving broadcasting and tensor shapes.

Is using GPT in this way cheating? It can be, if your first instinct is to jump to GPT rather than trying to understand the code yourself. But it's important here to bring up the distinction of [playing in easy mode](https://www.lesswrong.com/posts/nJPtHHq6L7MAMBvRK/play-in-easy-mode) vs [playing in hard mode](https://www.lesswrong.com/posts/7hLWZf6kFkduecH2g/play-in-hard-mode). There are situations where it's valuable for you to think about a problem for a while before moving forward because that deliberation will directly lead to you becoming a better researcher or engineer (e.g. when you're thinking of a hypothesis for how a circuit works while doing mechanistic interpretability on a transformer, or you're pondering which datastructure best fits your use case while implementing some RL algorithm). But there are also situations (like this one) where you'll get more value from speedrunning towards an understanding of certain code or concepts, and apply your understanding in subsequent exercises. It's important to find a balance!

#### When to use GPT-3.5 and GPT-4

GPT-3.5 and 4 both have advantages and disadvantages in different situations. GPT-3.5 has a large advantage in speed over GPT-4, and works equally well on simple problems or functions. If it's anything that Copilot is capable of writing, then you're likely better off using it instead of GPT-4.

On the other hand, GPT-4 has an advantage at generating coherent code (although we don't expect you to be using it for code generation much at this stage in the program), and is generally better at responding to complex tasks with less prompt engineering.

#### Additional notes on using GPT (from Joseph Bloom)

* ChatGPT is overly friendly. If you give it bad code it won't tell you it's shit so you need to encourage it to give feedback and/or show you examples of great code. Especially for beginner coders using it, it's important to realise how *under* critical it is.
* GPT is great at writing tests (asking it to write a test for a function is often better than asking it if a function is correct), refactoring code (identifying repeated tasks and extracting them) and naming variables well. These are specific things worth doing a few times to see how useful they can be.
* GPT-4 does well with whole modules/scripts so don't hesitate to add those. When you start managing repos on GitHub, use tracked files so that when you copy-paste edited code back, all the changes are highlighted for you as if you'd made them. (VSCode highlights changes with small blue bars next to the line numbers in your python files.)

Here are some things you can play around with:

* Ask GPT it to write tests for the function. You can give more specific instructions (e.g. asking it to use / not to use the `unittests` library, or to print more informative error messages).
* Ask GPT how to refactor the function above. (When I did this, it suggested splitting the function up into subfunctions which performed the discrete tasks of "compute intersection points")


## 2D Rays

Now we're going to make use of the z dimension and have rays emitted from the origin in both y and z dimensions.


### Exercise - implement `make_rays_2d`

```yaml
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

Implement `make_rays_2d` analogously to `make_rays_1d`. The result should look like a pyramid with the tip at the origin.


In [118]:
@jaxtyped
@typeguard.typechecked
def make_rays_2d(num_pixels_y: int, num_pixels_z: int, y_limit: float, z_limit: float) -> Float[t.Tensor, "nrays 2 3"]:
    '''
    num_pixels_y: The number of pixels in the y dimension
    num_pixels_z: The number of pixels in the z dimension

    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both.
    z_limit: At x=1, the rays should extend from -z_limit to +z_limit, inclusive of both.

    Returns: shape (num_rays=num_pixels_y * num_pixels_z, num_points=2, num_dims=3).
    '''
    output = t.zeros((num_pixels_y, num_pixels_z, 2, 3))
    output[..., 1, 0] = 1
    ys = t.linspace(-y_limit, y_limit, num_pixels_y)
    ys = einops.rearrange(ys, "h -> h 1")
    output[..., 1, 1] = ys
    output[..., 1, 2] = t.linspace(-z_limit, z_limit, num_pixels_z)
    output = einops.rearrange(output, "y z p d -> (y z) p d")
    return output

rays_2d = make_rays_2d(10, 10, 0.3, 0.3)
render_lines_with_plotly(rays_2d)

<details>
<summary>Help - I'm not sure how to implement this function.</summary>

Don't write it as a function right away. The most efficient way is to write and test each line individually in the REPL to verify it does what you expect before proceeding.

You can either build up the output tensor using `torch.stack`, or you can initialize the output tensor to its final size and then assign to slices like `rays[:, 1, 1] = ...`. It's good practice to be able to do it both ways.

Each y coordinate needs a ray with each corresponding z coordinate - in other words this is an outer product. The most elegant way to do this is with two calls to `einops.repeat`. You can also accomplish this with `unsqueeze`, `expand`, and `reshape` combined.
</details>

## Triangle Coordinates

The area inside a triangle can be defined by three (non-collinear) points $A$, $B$ and $C$, and can be written algebraically as a **convex combination** of those three points:

$$
\begin{align*}
P(w, u, v) &= wA + uB + vC \quad\quad \\
    \\
s.t. \quad 0 &\leq w,u,v \\
1 &= w + u + v
\end{align*}
$$

Or equivalently:

$$
\begin{align*}
\quad\quad\quad\quad P(u, v) &= (1 - u - v)A + uB + vC \\
&= A + u(B - A) + v(C - A) \\
\\
s.t. \quad 0 &\leq u,v \\
u + v &\leq 1
\end{align*}
$$

These $u, v$ are called "barycentric coordinates".

If we remove the bounds on $u$ and $v$, we get an equation for the plane containing the triangle. Play with the widget to understand the behavior of $u, v$.


In [119]:
one_triangle = t.tensor([[0, 0, 0], [3, 0.5, 0], [2, 3, 0]])
A, B, C = one_triangle
x, y, z = one_triangle.T

fig = setup_widget_fig_triangle(x, y, z)

@interact(u=(-0.5, 1.5, 0.01), v=(-0.5, 1.5, 0.01))
def response(u=0.0, v=0.0):
    P = A + u * (B - A) + v * (C - A)
    fig.data[2].update({"x": [P[0]], "y": [P[1]]})

display(fig)

interactive(children=(FloatSlider(value=0.0, description='u', max=1.5, min=-0.5, step=0.01), FloatSlider(value…

FigureWidget({
    'data': [{'marker': {'size': 12},
              'mode': 'markers+text',
              'text': [A, B, C],
              'textfont': {'size': 18},
              'textposition': 'middle left',
              'type': 'scatter',
              'uid': '59c6b5bf-2d57-4a37-b829-e848e2583204',
              'x': array([0., 3., 2.], dtype=float32),
              'y': array([0. , 0.5, 3. ], dtype=float32)},
             {'mode': 'lines',
              'type': 'scatter',
              'uid': '6bb2a321-5201-4275-b4cf-05a431676a22',
              'x': [0.0, 3.0, 2.0, 0.0],
              'y': [0.0, 0.5, 3.0, 0.0]},
             {'marker': {'size': 12, 'symbol': 'x'},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '7c182051-8582-46be-b99c-2c2cc38b4d73',
              'x': [0.0],
              'y': [0.0]}],
    'layout': {'height': 600,
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Barycen

### Triangle-Ray Intersection

Given a ray with origin $O$ and direction $D$, our intersection algorithm will consist of two steps:

- Finding the intersection between the line and the plane containing the triangle, by solving the equation $P(u, v) = P(s)$;
- Checking if $u$ and $v$ are within the bounds of the triangle.

Expanding the equation $P(u, v) = P(s)$, we have:

$$
\begin{align*}
A + u(B - A) + v(C - A) &= O + sD \\
\Rightarrow
\begin{pmatrix}
    -D & (B - A) & (C - A) \\
\end{pmatrix}
\begin{pmatrix}
    s \\
    u \\
    v  
\end{pmatrix}
&= \begin{pmatrix} O - A \end{pmatrix} \\
\Rightarrow \begin{pmatrix}
    -D_x & (B - A)_x & (C - A)_x \\
    -D_y & (B - A)_y & (C - A)_y \\
    -D_z & (B - A)_z & (C - A)_z \\
\end{pmatrix}
\begin{pmatrix}
    s \\
    u \\
    v  
\end{pmatrix} &= \begin{pmatrix}
    (O - A)_x \\
    (O - A)_y \\
    (O - A)_z \\
\end{pmatrix}
\end{align*}
$$

$$
$$

We can therefore find the coordinates `s`, `u`, `v` of the intersection point by solving the linear system above.


### Exercise - implement `triangle_ray_intersects`

```yaml
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 15-20 minutes on this exercise.
```

Using `torch.linalg.solve` and `torch.stack`, implement `triangle_ray_intersects(A, B, C, O, D)`.

A few tips:

* If you have a 0-dimensional tensor with shape `()` containing a single value, use the `item()` method to convert it to a plain Python value.
* If you have a tensor of shape `tensor.shape = (3, ...)`, then you can unpack it along the first dimension into three separate tensors the same way that you'd unpack a normal python list: `s, u, v = tensor`.
    * Note - if the dimension you want to unpack isn't at the start, a nice alternative is `s, u, v = tensor.unbind(dim)`, which does the same thing but along the dimension given by `dim` rather than the first dimension.
* If your function isn't working, try making a simple ray and triangle with nice round numbers where you can work out manually if it should intersect or not, then debug from there.


In [145]:
Point = Float[Tensor, "points=3"]

@jaxtyped
@typeguard.typechecked
def triangle_ray_intersects(A: Point, B: Point, C: Point, O: Point, D: Point) -> bool:
    '''
    A: shape (3,), one vertex of the triangle
    B: shape (3,), second vertex of the triangle
    C: shape (3,), third vertex of the triangle
    O: shape (3,), origin point
    D: shape (3,), direction point

    Return True if the ray and the triangle intersect.
    '''
    lhs = t.stack([-D, B-A, C-A], dim=1)
    rhs = t.stack([O - A], dim=1)
    try:
        x = t.linalg.solve(lhs, rhs)
        return bool((x[1] >= 0) and (x[2] >= 0) and (x[1] + x[2] <= 1))
    except Exception as e:
        return False

tests.test_triangle_ray_intersects(triangle_ray_intersects)

All tests in `test_triangle_ray_intersects` passed!


## Single-Triangle Rendering

Implement `raytrace_triangle` using only one call to `torch.linalg.solve`.

Reshape the output and visualize with `plt.imshow`. It's normal for the edges to look pixelated and jagged - using a small number of pixels is a good way to debug quickly.

If you think it's working, increase the number of pixels and verify that it looks less pixelated at higher resolution.


### Views and Copies

It's critical to know when you are making a copy of a `Tensor`, versus making a view of it that shares the data with the original tensor. It's preferable to use a view whenever possible to avoid copying memory unnecessarily. On the other hand, modifying a view modifies the original tensor which can be unintended and surprising. Consult [the documentation](https://pytorch.org/docs/stable/tensor_view.html) if you're unsure if a function returns a view. A short reference of common functions:

- `torch.expand`: always returns a view
- `torch.view`: always returns a view
- `torch.detach`: always returns a view
- `torch.repeat`: always copies
- `torch.clone`: always copies
- `torch.flip`: always copies (different than numpy.flip which returns a view)
- `torch.tensor`: always copies, but PyTorch recommends using `.clone().detach()` instead.
- `torch.Tensor.contiguous`: returns self if possible, otherwise a copy
- `torch.transpose`: returns a view if possible, otherwise (sparse tensor) a copy
- `torch.reshape`: returns a view if possible, otherwise a copy
- `torch.flatten`: returns a view if possible, otherwise a copy (different than numpy.flatten which returns a copy)
- `einops.repeat`: returns a view if possible, otherwise a copy
- `einops.rearrange`: returns a view if possible, otherwise a copy
- Basic indexing returns a view, while advanced indexing returns a copy.


### Storage Objects

Calling `storage()` on a `Tensor` returns a Python object wrapping the underlying C++ array. This array is 1D regardless of the dimensionality of the `Tensor`. This allows you to look inside the `Tensor` abstraction and see how the actual data is laid out in RAM.

Note that a new Python wrapper object is generated each time you call `storage()`, and both `x.storage() == x.storage()` and `x.storage() is x.storage()` evaluates to False.

If you want to check if two `Tensor`s share an underlying C++ array, you can compare their `storage().data_ptr()` fields. This can be useful for debugging.


### `Tensor._base`

If `x` is a view, you can access the original `Tensor` with `x._base`. This is an undocumented internal feature that's useful to know. Consider the following code:

```python
x = t.zeros(1024*1024*1024)
y = x[0]
del x
```

Here, `y` was created through basic indexing, so `y` is a view and `y._base` refers to `x`. This means `del x` won't actually deallocate the 4GB of memory, and that memory will remain in use which can be quite surprising. `y = x[0].clone()` would be an alternative here that does allow reclaiming the memory.


### Exercise - implement `raytrace_triangle`

```yaml
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 15-20 minutes on this exercise.

This is about as hard as `intersect_rays_1d`, although hopefully you should find it more familiar.
```

Below, you should implement `raytrace_triangle`, a funtion which checks whether each ray in `ray` intersects a single given triangle.


In [158]:
@jaxtyped
@typeguard.typechecked
def raytrace_triangle(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"],
    triangle: Float[Tensor, "trianglePoints=3 dims=3"]
) -> Bool[Tensor, "nrays"]:
    '''
    For each ray, return True if the triangle intersects that ray.
    '''
    triangles = einops.repeat(triangle, "p d -> n p d", n = rays.shape[0])
    lhs = t.stack([-rays[:,1,:], triangles[:,1,:] - triangles[:,0,:], triangles[:,2,:] - triangles[:,0,:]], dim=2)
    rhs = t.stack([rays[:,0,:] - triangles[:,0,:]], dim=2)
    xs = t.linalg.solve(lhs, rhs)
    output = (xs[:,1,:] >= 0) & (xs[:,2,:] >= 0) & (xs[:,1,:] + xs[:,2,:] <= 1)
    return einops.rearrange(output, "n 1 -> n")
    

A = t.tensor([1, 0.0, -0.5])
B = t.tensor([1, -0.5, 0.0])
C = t.tensor([1, 0.5, 0.5])
num_pixels_y = num_pixels_z = 15
y_limit = z_limit = 0.5

# Plot triangle & rays
test_triangle = t.stack([A, B, C], dim=0)
rays2d = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
triangle_lines = t.stack([A, B, C, A, B, C], dim=0).reshape(-1, 2, 3)
render_lines_with_plotly(rays2d, triangle_lines)

# Calculate and display intersections
intersects = raytrace_triangle(rays2d, test_triangle)
img = intersects.reshape(num_pixels_y, num_pixels_z).int()
imshow(img, origin="lower", width=600, title="Triangle (as intersected by rays)")

## Debugging


Debugging code is an extremely important thing to learn. Just like using GPT to assist your coding, it's something that can significantly speedrun your development, and stop you getting hung up on all the frustrating things!

To give you practice debugging using VSCode's built-in debugger\*, we've provided an example function below. This is an implementation of `raytrace_triangle` which has a few mistakes in it. Your task is to use the debugger to find the mistake and fix it. (Note - you're encouraged to actually use the debugger, rather than comparing it to the solution above!)


In [ ]:
# Incorrect function:

def raytrace_triangle_with_bug(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"],
    triangle: Float[Tensor, "trianglePoints=3 dims=3"]
) -> Bool[Tensor, "nrays"]:
    '''
    For each ray, return True if the triangle intersects that ray.
    '''
    NR = rays.size[0]

    A, B, C = einops.repeat(triangle, "pts dims -> pts NR dims", NR=NR)

    O, D = rays.unbind(-1)

    mat = t.stack([- D, B - A, C - A])

    dets = t.linalg.det(mat)
    is_singular = dets.abs() < 1e-8
    mat[is_singular] = t.eye(3)

    vec = O - A

    sol = t.linalg.solve(mat, vec)
    s, u, v = sol.unbind(dim=-1)

    return ((u >= 0) & (v >= 0) & (u + v <= 1) & ~is_singular)


intersects = raytrace_triangle_with_bug(rays2d, test_triangle)
img = intersects.reshape(num_pixels_y, num_pixels_z).int()
imshow(img, origin="lower", width=600, title="Triangle (as intersected by rays)")


You can debug a cell by clicking on the **Debug cell** option at the bottom of your cell. Your cell should contain the code that is actually run to cause an error (rather than needing to contain the function which is the source of the error). Before you run your debugger, you can set breakpoints by clicking on the left-hand side of the line number (a red dot will appear). You can then step through your code, using the toolbar of buttons which will appear when you run the debugger (see [here](https://pawelgrzybek.com/continue-step-over-step-into-and-step-out-actions-in-visual-studio-code-debugger-explained/) for an explanation of what each of the buttons does). When you reach a breakpoint, you can use the following tools:

* Inspect local and global variables, in the **VARIABLES** window that appears on the left sidebar.
* Add variables expressions to watch, in the **WATCH** window that appears on the left sidebar.
    * You can just type in any expression here, e.g. the type of a variable or the length of a list, and this will update as you step through the function.
* Evaluate expressions on a one-time basis, by typing them into the **DEBUG CONSOLE** (which appears at the bottom of the screen)

Note, when you run the debugger, it will stop *before* the breakpoint line is evaluated. So if you've run a cell and got an error on a specific line, then *that* is the line you want to set a breakpoint on.

*This all works basically the same if you're in a notebook in VSCode, except for a few changes e.g. the debug button is on the dropdown at the top-left of the cell (if it doesn't appear for you then you'll need to go into user settings and add the line `"notebook.consolidatedRunButton": true`).*

There's much more detail we could go into about debugging, but this will suffice for most purposes. It's generally a much more efficient way of debugging than using print statements or asserts (although these can also be helpful in some situations).


<details>
<summary>Answer - what are the bugs, and how can they be fixed?</summary>

```python
NR = rays.size[0]
```

This should be `rays.size(0)` (or equivalently, `rays.shape[0]`). `size` is a method which takes an int and returns the size of that dimension; `shape` returns a `torch.Size` object which can be indexed into.

This would have been pretty easy to realise without using the debugger, because the error message is informative.

```python
O, D = rays.unbind(-1)
```

We're unbinding from the wrong dimension. `rays` has shape `(nrays, points=2, dims=3)`, and we actually want to unbind along the `points` dimension. So we should use `rays.unbind(1)`.

We could have found this by inspecting the `rays` object in the variables pane (you can click on the dropdown next to the variable name to look at its attributes, including `shape`), or you could just type `rays.shape` into the debug console. However, this error should also have been apparent given the type signature of the function (a good case for using typing!).

```python
mat = t.stack([- D, B - A, C - A])
```

This should have had the argument `dim=-1` on the end, because `torch.stack` by default stacks along the first dimension.

This error is a bit harder to diagnose, because the line causing an error is after the line containing the mistake. Again, we could have diagnosed this with the debugger by inspecting the shape of our `mat` tensor in the variables pane.

---

These were all relatively easy bugs to diagnose (not all bugs will present as actual errors in your code, some might just be an unexpected set of results). But hopefully this has given you a sense for how to use the debugger. It's a very powerful tool, and can save you a lot of time!
</details>


\**If you're using Colab, you'll have to use different strategies for debugging. See [this page](https://zohaib.me/debugging-in-google-collab-notebook/) for one suggested approach.*


## Mesh Loading

Use the given code to load the triangles for your Pikachu. By convention, files written with `torch.save` end in the `.pt` extension, but these are actually just zip files.


In [161]:
with open(current_dir / "pikachu.pt", "rb") as f:
    triangles = t.load(f)

## Mesh Rendering

For our purposes, a mesh is just a group of triangles, so to render it we'll intersect all rays and all triangles at once. We previously just returned a boolean for whether a given ray intersects the triangle, but now it's possible that more than one triangle intersects a given ray.

For each ray (pixel) we will return a float representing the minimum distance to a triangle if applicable, otherwise the special value `float('inf')` representing infinity. We won't return which triangle was intersected for now.


### Exercise - implement `raytrace_mesh`

```yaml
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 20-25 minutes on this exercise.

This is the main function we've been building towards, and marks the end of the core exercises. If should involve a lot of repurposed code from the last excercise.
```

Implement `raytrace_mesh` and as before, reshape and visualize the output. Your Pikachu is centered on (0, 0, 0), so you'll want to slide the ray origin back to at least `x=-2` to see it properly.

Reminder - `t.linalg.solve` (and most batched operations) can accept multiple dimensions as being batch dims. Previously, you've just used `NR` (the number of rays) as the batch dimension, but you can also use `(NR, NT)` (the number of rays and triangles) as your batch dimensions, so you can solve for all rays and triangles at once.

Note - by distance to a triangle, we specifically mean **distance along the x-dimension**, not Euclidean distance.

In [185]:
def raytrace_mesh(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"],
    triangles: Float[Tensor, "ntriangles trianglePoints=3 dims=3"]
) -> Float[Tensor, "nrays"]:
    '''
    For each ray, return the distance to the closest intersecting triangle, or infinity.
    '''
    rays_repeated = einops.repeat(rays, "nrays rp d -> nrays ntriangles rp d", ntriangles = triangles.shape[0])
    triangles_repeated = einops.repeat(triangles, "ntriangles tp d -> nrays ntriangles tp d", nrays = rays.shape[0])
    A = triangles_repeated[...,0,:]
    B = triangles_repeated[...,1,:]
    C = triangles_repeated[...,2,:]
    O = rays_repeated[...,0,:]
    D = rays_repeated[...,1,:]
    lhs = t.stack([-D, B - A, C - A], dim=-1)
    rhs = t.stack([O - A], dim=-1)
    dets = t.linalg.det(lhs)
    mask = dets.abs() < 1e-10
    lhs[mask] = t.eye(3)
    xs = t.linalg.solve(lhs, rhs)
    assert(xs.shape[3] == 1)
    xs = einops.reduce(xs, "nr nt d sq -> nr nt d", "min")
    valid = (xs[:,:,1] >= 0) & (xs[:,:,2] >= 0) & (xs[:,:,1] + xs[:,:,2] <= 1) & ~mask
    dists = xs[:,:,0]
    dists[~valid] = float("inf")
    return dists.min(dim=-1).values

num_pixels_y = 120
num_pixels_z = 120
y_limit = z_limit = 1

rays = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
rays[:, 0] = t.tensor([-2, 0.0, 0.0])
dists = raytrace_mesh(rays, triangles)
intersects = t.isfinite(dists).view(num_pixels_y, num_pixels_z)
dists_square = dists.view(num_pixels_y, num_pixels_z)
img = t.stack([intersects, dists_square], dim=0)

fig = px.imshow(img, facet_col=0, origin="lower", color_continuous_scale="magma", width=1000)
fig.update_layout(coloraxis_showscale=False)
for i, text in enumerate(["Intersects", "Distance"]):
    fig.layout.annotations[i]['text'] = text
fig.show()

Congratulations, you've now got to the end of the exercises! Read on for some more bonus material.

## Bonus - more raytracing



Here are a few bonus exercises for you to do with your time.

- Vectorize further to make a video.
    - Each frame will have its own rays coming from a slightly different position.
    - Pan the camera around for some dramatic footage.
    - One way to do it is using the `mediapy` library to render the video.
- Try rendering on the GPU and see if you can make it faster.
- Allow each triangle to have a corresponding RGB color value and render a colored image.
- Use multiple rays per pixel and combine them somehow to have smoother edges.

Since this is a very difficult exercise, it would be a great opportunity for you to wheel out GPT-4 and use it to generate code!

We've also provided you with a handful of sample renders below.

<details>
<summary>Render animation (using Plotly)</summary>

```python
from typing import Callable
from tqdm import tqdm

def raytrace_mesh_video(
    rays: TT["nrays", "points": 2, "ndims": 3],
    triangles: TT["ntriangles", "npoints": 3, "ndims": 3],
    rotation_matrix: Callable[[float], TT[3, 3]],
    num_frames: int,
) -> TT["nframes", "nrays", bool]:
    result = []
    theta = t.tensor(2 * t.pi) / num_frames
    R = rotation_matrix(theta)
    for theta in tqdm(range(num_frames)):
        triangles = triangles @ R
        result.append(raytrace_mesh(rays, triangles))
    return t.stack(result, dim=0)

if MAIN:
    num_pixels_y = 200
    num_pixels_z = 200
    y_limit = z_limit = 1
    num_frames = 50

    rotation_matrix = lambda theta: t.tensor([
        [t.cos(theta), 0.0, t.sin(theta)],
        [0.0, 1.0, 0.0],
        [-t.sin(theta), 0.0, t.cos(theta)],
    ])

    rays = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
    rays[:, 0] = t.tensor([-2, 0.0, 0.0])
    dists = raytrace_mesh_video(rays, triangles, rotation_matrix, num_frames)
    dists_square = dists.view(num_frames, num_pixels_y, num_pixels_z)

    fig = px.imshow(dists_square, animation_frame=0, origin="lower", color_continuous_scale="viridis_r")
    # zmin=0, zmax=2, color_continuous_scale="Brwnyl"
    fig.update_layout(coloraxis_showscale=False)
    fig.show()
```
</details>


<details>
<summary>Rendering on GPU</summary>

```python
from einops import repeat, reduce, rearrange

def make_rays_2d_origin(
    num_pixels_y: int, num_pixels_z: int, y_limit: float, z_limit: float, origin: t.Tensor
) -> t.Tensor:
    '''
    num_pixels_y: The number of pixels in the y dimension
    num_pixels_z: The number of pixels in the z dimension
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both.
    z_limit: At x=1, the rays should extend from -z_limit to +z_limit, inclusive of both.
    Returns: shape (num_rays=num_pixels_y * num_pixels_z, num_points=2, num_dims=3).
    '''
    rays = t.zeros((num_pixels_y, num_pixels_z, 2, 3))
    rays[:, :, 1, 0] = 1
    rays[:, :, 1, 1] = repeat(
        t.arange(num_pixels_y) * 2.0 * y_limit / (num_pixels_y - 1) - y_limit, "y -> y z", z=num_pixels_z
    )
    rays[:, :, 1, 2] = repeat(
        t.arange(num_pixels_z) * 2.0 * z_limit / (num_pixels_z - 1) - z_limit, "z -> y z", y=num_pixels_y
    )
    rays[:, :, 0, :] = origin
    return rearrange(rays, "y z n d -> (y z) n d", n=2, d=3)

def raytrace_mesh_gpu(triangles: t.Tensor, rays: t.Tensor) -> t.Tensor:
    '''For each ray, return the distance to the closest intersecting triangle, or infinity.
    triangles: shape (n_triangles, n_points=3, n_dims=3)
    rays: shape (n_pixels, n_points=2, n_dims=3)
    return: shape (n_pixels, )
    '''
    n_triangles = triangles.size(0)
    n_pixels = rays.size(0)
    device = "cuda"
    matrices = t.zeros((n_pixels, n_triangles, 3, 3)).to(device)
    rays_gpu = rays.to(device)
    matrices[:, :, :, 0] = repeat(rays_gpu[:, 0] - rays_gpu[:, 1], "r d -> r t d", t=n_triangles)
    triangles_gpu = triangles.to(device)
    matrices[:, :, :, 1] = repeat(triangles_gpu[:, 1] - triangles_gpu[:, 0], "t d -> r t d", r=n_pixels)
    matrices[:, :, :, 2] = repeat(triangles_gpu[:, 2] - triangles_gpu[:, 0], "t d -> r t d", r=n_pixels)
    bs = repeat(rays_gpu[:, 0], "r d -> r t d", t=n_triangles) - repeat(
        triangles_gpu[:, 0], "t d -> r t d", r=n_pixels
    )
    mask = t.linalg.det(matrices) != 0
    distances = t.full((n_pixels, n_triangles), float("inf")).to(device)
    solns = t.linalg.solve(matrices[mask], bs[mask])
    distances[mask] = t.where(
        (solns[:, 0] >= 0) & (solns[:, 1] >= 0) & (solns[:, 2] >= 0) & (solns[:, 1] + solns[:, 2] <= 1),
        solns[:, 0],
        t.tensor(float("inf")).to(device),
    )
    return reduce(distances, "r t -> r", "min").to("cpu")


if MAIN:
    num_pixels_y = 120
    num_pixels_z = 120
    y_limit = z_limit = 3
    rays = make_rays_2d_origin(num_pixels_y, num_pixels_z, y_limit, z_limit, t.tensor([-3.0, 0, 0]))
    intersections = raytrace_mesh_gpu(triangles, rays)
    picture = rearrange(intersections, "(y z) -> y z", y=num_pixels_y, z=num_pixels_z)
    
    fig = px.imshow(picture, origin="lower").update_layout(coloraxis_showscale=False)
    fig.show()
```
</details>

<details>
<summary>Rendering with lighting</summary>

```python
import math

def raytrace_mesh_lighting(
    triangles: t.Tensor, rays: t.Tensor, light: t.Tensor, ambient_intensity: float, device: str = "cpu"
) -> t.Tensor:
    '''For each ray, return the shade of the nearest triangle.
    triangles: shape (n_triangles, n_points=3, n_dims=3)
    rays: shape (n_pixels, n_points=2, n_dims=3)
    light: shape (n_dims=3, )
    device: The device to place tensors on.
    return: shape (n_pixels, )
    '''
    n_triangles = triangles.size(0)
    n_pixels = rays.size(0)
    triangles = triangles.to(device)
    rays = rays.to(device)
    light = light.to(device)

    matrices = t.zeros((n_pixels, n_triangles, 3, 3)).to(device)
    directions = rays[:, 1] - rays[:, 0]
    matrices[:, :, :, 0] = repeat(-directions, "r d -> r t d", t=n_triangles)
    matrices[:, :, :, 1] = repeat(triangles[:, 1] - triangles[:, 0], "t d -> r t d", r=n_pixels)
    matrices[:, :, :, 2] = repeat(triangles[:, 2] - triangles[:, 0], "t d -> r t d", r=n_pixels)
    bs = repeat(rays[:, 0], "r d -> r t d", t=n_triangles) - repeat(triangles[:, 0], "t d -> r t d", r=n_pixels)
    mask = t.linalg.det(matrices) != 0
    distances = t.full((n_pixels, n_triangles), float("inf")).to(device)
    solns = t.linalg.solve(matrices[mask], bs[mask])
    distances[mask] = t.where(
        (solns[:, 0] >= 0) & (solns[:, 1] >= 0) & (solns[:, 2] >= 0) & (solns[:, 1] + solns[:, 2] <= 1),
        solns[:, 0],
        t.tensor(float("inf")).to(device),
    )
    closest_triangle = distances.argmin(1)

    normals = t.cross(triangles[:, 1] - triangles[:, 0], triangles[:, 2] - triangles[:, 0], dim=1)
    normals = t.nn.functional.normalize(normals, p=2.0, dim=1)
    intensity = t.einsum("td,d->t", normals, light).gather(0, closest_triangle)
    side = t.einsum("rd,rd->r", normals.gather(0, repeat(closest_triangle, "r -> r d", d=3)), directions)
    intensity = t.maximum(t.sign(side) * intensity, t.zeros(())) + ambient_intensity
    intensity = t.where(
        distances.gather(1, closest_triangle.unsqueeze(1)).squeeze(1) == float("inf"),
        t.tensor(0.0).to(device),
        intensity,
    )

    return intensity.to("cpu")

def make_rays_camera(
    num_pixels_v: int,
    num_pixels_w: int,
    v_limit: float,
    w_limit: float,
    origin: t.Tensor,
    screen_distance: float,
    roll: float,
    pitch: float,
    yaw: float,
) -> t.Tensor:
    '''
    num_pixels_y: The number of pixels in the y dimension
    num_pixels_z: The number of pixels in the z dimension
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both.
    z_limit: At x=1, the rays should extend from -z_limit to +z_limit, inclusive of both.
    Returns: shape (num_rays=num_pixels_y * num_pixels_z, num_points=2, num_dims=3).
    '''

    normal = t.tensor([math.cos(pitch) * math.cos(yaw), math.sin(pitch), math.cos(pitch) * math.sin(yaw)])
    w_vec = t.nn.functional.normalize(t.tensor([normal[2], 0, -normal[0]]), p=2.0, dim=0)
    v_vec = t.cross(normal, w_vec)
    w_vec_r = math.cos(roll) * w_vec + math.sin(roll) * v_vec
    v_vec_r = math.cos(roll) * v_vec - math.sin(roll) * w_vec

    rays = t.zeros((num_pixels_y, num_pixels_z, 2, 3))
    rays[:, :, 1, :] += repeat(origin + normal * screen_distance, "d -> w v d", w=num_pixels_w, v=num_pixels_v)
    rays[:, :, 1, :] += repeat(
        t.einsum("w, d -> w d", (t.arange(num_pixels_w) * 2.0 * w_limit / (num_pixels_w - 1) - w_limit), w_vec_r),
        "w d -> w v d",
        v=num_pixels_v,
    )
    rays[:, :, 1, :] += repeat(
        t.einsum("v, d -> v d", t.arange(num_pixels_v) * 2.0 * v_limit / (num_pixels_v - 1) - v_limit, v_vec_r),
        "v d -> w v d",
        w=num_pixels_w,
    )

    rays[:, :, 0, :] = origin
    return rearrange(rays, "y z n d -> (y z) n d", n=2, d=3)



if MAIN:
    num_pixels_y = 150
    num_pixels_z = 150
    y_limit = z_limit = 3
    rays = make_rays_2d_origin(num_pixels_y, num_pixels_z, y_limit, z_limit, t.tensor([-3.0, 0, 0]))
    light = t.tensor([0.0, -1.0, 1.0])
    ambient_intensity = 0.5
    intersections = raytrace_mesh_lighting(triangles, rays, light, ambient_intensity, "cuda")
    picture = rearrange(intersections, "(y z) -> y z", y=num_pixels_y, z=num_pixels_z)
    
    fig = px.imshow(picture, origin="lower", labels={"x": "X", "y": "Y"}, color_continuous_scale="magma").update_layout(coloraxis_showscale=False)
    fig.show()
```

```python
def get_random_rotation_matrix(N, theta_max=t.pi):
    mat = t.eye(N)
    for i in range(N):
        rot_mat = t.eye(N)
        theta = (t.rand(1) - 0.5) * theta_max
        rot_mat_2d = t.tensor([
            [t.cos(theta), -t.sin(theta)],
            [t.sin(theta), t.cos(theta)]
        ])
        if i == N - 1:
            rot_mat[[-1, -1, 0, 0], [-1, 0, -1, 0]] = rot_mat_2d.flatten()
        else:
            rot_mat[i :i+2, i :i+2] = rot_mat_2d
        mat = mat @ rot_mat
    return mat

if MAIN:
    num_pixels_y = 150
    num_pixels_z = 150
    y_limit = z_limit = 3

    rays = make_rays_camera(num_pixels_y, num_pixels_z, y_limit, z_limit, t.tensor([-1.0, 3.0, 0.0]), 3.0, 0.0, -1.0, 0.0)
    light = t.tensor([0.0, -1.0, 1.0])
    ambient_intensity = 0.5
    intersections = raytrace_mesh_lighting(triangles, rays, light, ambient_intensity, "cuda")
    picture = rearrange(intersections, "(y z) -> y z", y=num_pixels_y, z=num_pixels_z)
    fig = px.imshow(picture, origin="lower", labels={"x": "X", "y": "Y"}, color_continuous_scale="magma").update_layout(coloraxis_showscale=False)
    fig.show()
```
</details>

<details>
<summary>Rendering with lambert shading</summary>

```python
def raytrace_mesh_lambert(triangles: t.Tensor, rays: t.Tensor) -> t.Tensor:
    '''For each ray, return the distance to the closest intersecting triangle, or infinity.
    triangles: shape (n_triangles, n_points=3, n_dims=3)
    rays: shape (n_pixels, n_points=2, n_dims=3)
    return: shape (n_pixels, )
    '''
    # triangles = [triangle, point, coord]
    # rays = [pixel, orig_dir, coord]

    n_triangles = len(triangles)
    n_pixels = len(rays)

    rep_triangles = einops.repeat(triangles, "triangle point coord -> pixel triangle point coord", pixel=n_pixels)
    rep_rays = einops.repeat(rays, "pixel orig_dir coord -> pixel triangle orig_dir coord", triangle=n_triangles)

    O = rep_rays[:, :, 0, :]  # [pixel, triangle, coord]
    D = rep_rays[:, :, 1, :]  # [pixel, triangle, coord]
    A = rep_triangles[:, :, 0, :]  # [pixel, triangle, coord]
    B = rep_triangles[:, :, 1, :]  # [pixel, triangle, coord]
    C = rep_triangles[:, :, 2, :]  # [pixel, triangle, coord]
    rhs = O - A  # [pixel, triangle, coord]
    lhs = t.stack([-D, B - A, C - A], dim=3)  # [pixel, triangle, coord, suv]
    dets = t.linalg.det(lhs)  # [pixel, triangle]
    dets = dets < 1e-5
    eyes = t.einsum("i j , k l -> i j k l", [dets, t.eye(3)])
    lhs += eyes
    results = t.linalg.solve(lhs, rhs)  # [pixel, triangle, suv]
    intersects = (
        ((results[:, :, 1] + results[:, :, 2]) <= 1)
        & (results[:, :, 0] >= 0)
        & (results[:, :, 1] >= 0)
        & (results[:, :, 2] >= 0)
        & (dets == False)
    )  # [pixel, triangle]
    distances = t.where(intersects, results[:, :, 0].double(), t.inf)  # [pixel, triangle]

    # Lambert shading (dot product of triangle's normal vector with light direction)
    indices = t.argmin(distances, dim=1)
    tri_vecs1 = triangles[:, 0, :] - triangles[:, 1, :]
    tri_vecs2 = triangles[:, 1, :] - triangles[:, 2, :]
    normvecs = t.cross(tri_vecs1, tri_vecs2, dim=1)  # [triangle coord]
    normvecs -= normvecs.min(1, keepdim=True)[0]
    normvecs /= normvecs.max(1, keepdim=True)[0]
    lightvec = t.tensor([[0.0, 1.0, 1.0]] * n_triangles)
    tri_lights = abs(t.einsum("t c , t c -> t", [normvecs, lightvec]))  # triangle
    pixel_lights = 1.0 / (einops.reduce(distances, "pixel triangle -> pixel", "min")) ** 2
    pixel_lights *= tri_lights[indices]
    return pixel_lights



if MAIN:
    rot_mat = get_random_rotation_matrix(N=3, theta_max=t.pi/3)
    num_pixels_y = 200
    num_pixels_z = 200
    y_limit = z_limit = 1
    rays = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
    rays[:, 0, 0] = -2
    rays[0, :, 0]
    result = raytrace_mesh_lambert(t.einsum("i j k, k l -> i j l", [triangles, rot_mat]), rays)
    result = result.reshape(num_pixels_y, num_pixels_z)
    fig = px.imshow(result, origin="lower", labels={"x": "X", "y": "Y"}, color_continuous_scale="magma").update_layout(coloraxis_showscale=False)
    fig.show()
```
</details>


<details>
<summary>Rendering with hollow triangles</summary>

```python
def raytrace_mesh_lambert_wireframe(triangles: t.Tensor, rays: t.Tensor, triangle_perim: float = 0) -> t.Tensor:
    '''For each ray, return the distance to the closest intersecting triangle, or infinity.
    triangles: shape (n_triangles, n_points=3, n_dims=3)
    rays: shape (n_pixels, n_points=2, n_dims=3)
    return: shape (n_pixels, )
    '''
    # triangles = [triangle, point, coord]
    # rays = [pixel, orig_dir, coord]

    n_triangles = len(triangles)
    n_pixels = len(rays)

    rep_triangles = einops.repeat(triangles, "triangle point coord -> pixel triangle point coord", pixel=n_pixels)
    rep_rays = einops.repeat(rays, "pixel orig_dir coord -> pixel triangle orig_dir coord", triangle=n_triangles)

    O = rep_rays[:, :, 0, :]  # [pixel, triangle, coord]
    D = rep_rays[:, :, 1, :]  # [pixel, triangle, coord]
    A = rep_triangles[:, :, 0, :]  # [pixel, triangle, coord]
    B = rep_triangles[:, :, 1, :]  # [pixel, triangle, coord]
    C = rep_triangles[:, :, 2, :]  # [pixel, triangle, coord]
    rhs = O - A  # [pixel, triangle, coord]
    lhs = t.stack([-D, B - A, C - A], dim=3)  # [pixel, triangle, coord, suv]
    dets = t.linalg.det(lhs)  # [pixel, triangle]
    dets = dets < 1e-5
    eyes = t.einsum("i j , k l -> i j k l", [dets, t.eye(3)])
    lhs += eyes
    results = t.linalg.solve(lhs, rhs)  # [pixel, triangle, suv]
    intersects = (
        ((results[:, :, 1] + results[:, :, 2]) <= 1)
        & (results[:, :, 0] >= 0.0)
        & (results[:, :, 1] >= 0.0)
        & (results[:, :, 2] >= 0.0)
        & (dets == False)
    )  # [pixel, triangle]
    intersects_perim = (
        ((results[:, :, 1] + results[:, :, 2]) >= 1 - triangle_perim)
        | (results[:, :, 1] <= triangle_perim)
        | (results[:, :, 2] <= triangle_perim)
    )
    intersects = intersects & intersects_perim
    distances = t.where(intersects, results[:, :, 0].double(), t.inf)  # [pixel, triangle]

    # Lambert shading (dot product of triangle's normal vector with light direction)
    indices = t.argmin(distances, dim=1)
    tri_vecs1 = triangles[:, 0, :] - triangles[:, 1, :]
    tri_vecs2 = triangles[:, 1, :] - triangles[:, 2, :]
    normvecs = t.cross(tri_vecs1, tri_vecs2, dim=1)  # [triangle coord]
    normvecs -= normvecs.min(1, keepdim=True)[0]
    normvecs /= normvecs.max(1, keepdim=True)[0]
    lightvec = t.tensor([[0.0, 1.0, 1.0]] * n_triangles)
    tri_lights = abs(t.einsum("t c , t c -> t", [normvecs, lightvec]))  # triangle
    pixel_lights = 1.0 / (einops.reduce(distances, "pixel triangle -> pixel", "min")) ** 2
    pixel_lights *= tri_lights[indices]
    return pixel_lights


if MAIN:
    rot_mat = get_random_rotation_matrix(N=3, theta_max=t.pi/4)
    num_pixels_y = 200
    num_pixels_z = 200
    y_limit = z_limit = 1

    triangle_perim = 0.1

    rays = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
    rays[:, 0, 0] = -2
    rays[0, :, 0]
    result = raytrace_mesh_lambert_wireframe(t.einsum("i j k, k l -> i j l", [triangles, rot_mat]), rays, triangle_perim)
    result = result.reshape(num_pixels_y, num_pixels_z)
    fig = px.imshow(result, origin="lower", labels={"x": "X", "y": "Y"}, color_continuous_scale="magma").update_layout(coloraxis_showscale=False)
    fig.show()
```
</details>


## Bonus - testing with `pytest`


> *Note - this section was also just written with VSCode in mind, so it might be less suitable for Colab, and you might consider skipping it.*

To wrap up today, we're going to have a further discussion of **testing**. So far, our test functions have been pretty simple - we imported a test from the `part1_ray_tracing/tests.py` file, and ran it to compare our answers to the answers in `part1_ray_tracing/solutions.py`. This works perfectly fine, but there are other Python libraries which can make testing easier and more powerful. In particular, two such libraries are `unittest` and `pytest`. Both these libraries provide extensive features for modularizing and running test functions, and have nice integrations with VSCode. `unittest` is very powerful, but also has a steep learning curve and isn't worth the trouble of learning during a 6 week course. However, `pytest` is very useful and easy to learn, so we'll spend some time here discussing it.

The pytest framework makes it easy to write small, readable tests, and can scale to support complex functional testing for applications and libraries. Today we'll just discuss a few features which will be particularly useful for the kinds of tests we'll be writing, but you're welcome to explore more tests later on.

*Important note - if you're working on Colab or a notebook for these exercises, your tests won't work, because in this setup we'll be importing answers from our test file, rather than importing tests from our answers file. For this reason, we won't be using pytest for the structured exercises in this course (in case anybody is following these exercises in a notebook), but you're strongly recommended to use pytest when you're working from `.py` files instead - which is strongly recommended during much of the bonus content.*

First, look at `part1_ray_tracing/tests.py`. The tests in this file all follow a similar pattern: evaluate your function on some input, check it equals the solution. This is a clear sign that we can use modularization to simplify our code! Compare this file to `part1_ray_tracing/test_with_pytest.py`, which contains the same tests but refactored using pytest. The latter is clearly separated into three sections: one for library imports, one for defining the inputs we'll use for our test functions, and one where we define our test functions.

### Parametrization

We use `pytest.mark.parametrize` to parametrize our tests. This means we can run the same test a bunch of different times on different inputs. In the non-pytest version, we did this by defining different functions (e.g. `test_intersect_ray_1d` and `test_intersect_ray_1d_special_case`), but this isn't necessary here.

The syntax of a function decorated by `pytest.mark.parametrize` is:

```python
@pytest.mark.parametrize("arg1, arg2, arg3", [
    (val1, val2, val3),
    (val4, val5, val6),
    ...
])
def test_function(arg1, arg2, arg3):
    ...
```

This will result in running `test_function` multiple times, the first time with `arg1=val1, arg2=val2, arg3=val3`, and the second time with `arg1=val4, arg2=val5, arg3=val6`, etc. This is a much better way to write tests, because it makes it easy for us to add new test cases without having to write new functions.

### How to run these tests?

There are three ways to run the above tests:

1. **From the terminal**

You can run them from the command line using the pytest command. Open your terminal in VSCode at the bottom of the screen, make sure you're in the directory `chapter0_fundamentals`, then enter the command `pytest`. This will search the current directory (or subdirectories) for any python files with the prefix `test_`, and within that file run all functions with the prefix `test_` (i.e. all the tests for these exercises).

You can also add more complicated commands, like:

* `pytest test_intersect_ray_1d` to specify the file to run tests from
* `pytest test_intersect_ray_1d::test_intersect_ray_1d` to specify the file and test function

2. **From your Python file**

To do this, you need to call `pytest.main` (make sure this is wrapped in an `if MAIN` block). The first argument of this function is a list of args to `pytest`. This has analogous syntax to the command line option, for instance here are a few ways you can do this:

```python
if MAIN:
    pytest.main([])
    pytest.main(["test_part1_ray_tracing.py"])
    pytest.main(["test_part1_ray_tracing.py::test_intersect_ray_1d"])
```

3. **From VSCode's testing display**

This is by far the most useful way to run test functions. VSCode provides a helpful interface to run all test functions in your directory, and see which of them have passed. First, press the test icon on the left hand sidebar (it will look like a triangular flask). If this is your first time, then you'll need to click on the **Configure Python Tests** button, then select `pytest` as your framework (you may have to install pytest when prompted). This should automatically perform test discovery for you, showing you all the test files & functions in your current directory in the left hand window (it detects this the same way pytest does, using the `test_` prefix). You can hover over tests or files to see options like "run test(s)" or debug test(s)". When you run them, you'll see a tick or cross appear next to the test, indicating whether it passed or failed.

This is a very useful interface when you're working on a large project, because as you make updates you can easily re-run all tests, then identify & fix problems.

<details>
<summary>Help - I get <code>Error discovering pytest tests</code></summary>

Try opening your terminal (at the bottom of the screen) and running `pytest --collect-only`. This should print output that helps you diagnose the problem.

One possible issue is changing directories within your python files (e.g. with `os.chdir`), because this can often confuse PyTest's search for tests.
</details>


<details>
<summary>Aside - <code># type: ignore</code></summary>

Note that the test file has the annotation `# type: ignore` at the end of an import line, at the top of the file. This is because VSCode's typechecker will complain when you try and import a file that doesn't exist yet! This line removes the type error, and allows you to write functions without all those annoying squiggly red lines! However, don't overuse this annotation, because it can often be a signal that you've made a mistake in your code, or else have followed a bad coding practice.

</details>

<details>
<summary>Aside - named tuples</summary>

Another way you can clean this code up is to use **named tuples**. These are an especially nice and Pythonic structure which maintains the features of a tuple, but also allows you to define instances & access elements by name. For instance, instead of:

```python
ray_segment_batch = (
    solutions.rays1d,
    solutions.segments
)
```

you could have:

```python
from collections import namedtuple
TestCase = namedtuple("TestCase", "rays segments")

ray_segment_batch = TestCase(
    rays = solutions.rays1d,
    segments = solutions.segments
)
```

and this `ray_segment_batch` object would still be treated like a tensor when it's used in a parameterized test.

</details>
